In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import cv2


# Load a pre-trained object detection model from TensorFlow Hub
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"
detector = hub.load(model_url)

# Function to load and preprocess image
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

# Function to draw bounding boxes
def draw_boxes(image, boxes, class_names, scores, threshold=0.5):
    image = np.array(image)
    height, width, _ = image.shape
    
    for i in range(boxes.shape[0]):
        if scores[i] >= threshold:
            ymin, xmin, ymax, xmax = boxes[i]
            xmin = int(xmin * width)
            xmax = int(xmax * width)
            ymin = int(ymin * height)
            ymax = int(ymax * height)
            
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            label = f"{class_names[i].decode('ascii')}: {scores[i]:.2f}"
            cv2.putText(image, label, (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
    return image

# Load an image
image_path = 'your_image.jpg'  # <-- replace with your test image path
image = load_image(image_path)
input_tensor = tf.expand_dims(image, 0)

# Run detection
detector_output = detector(input_tensor)

# Extract boxes, class names, and scores
boxes = detector_output['detection_boxes'][0].numpy()
class_names = detector_output['detection_class_entities'][0].numpy()
scores = detector_output['detection_scores'][0].numpy()

# Draw detections
result_img = draw_boxes(image, boxes, class_names, scores)

# Show the result
plt.figure(figsize=(10, 10))
plt.imshow(result_img)
plt.axis('off')
plt.show()


# Assignment 10

## Problem Statement:<br>
Write Python program to implement CNN object detection. Discuss numerous performance evaluation metrics for evaluating the object-detecting algorithms' performance.

### Import TensorFlow library

In [2]:
import tensorflow as tf

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### Preprocess the data

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0

### Add a channel dimension to the images

In [5]:
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

### Convert labels to one-hot encoding

In [6]:
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

### Define the CNN model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

### Compile the model

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [10]:
model.fit(train_images, train_labels, epochs=4, batch_size=32)

Epoch 1/4
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1642 - accuracy: 0.9516
Epoch 2/4
1875/1875 [==============================] - 29s 15ms/step - loss: 0.0565 - accuracy: 0.9826
Epoch 3/4
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0388 - accuracy: 0.9880
Epoch 4/4
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0272 - accuracy: 0.9913


### Evaluate the model

In [11]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

313/313 [==============================] - 2s 5ms/step - loss: 0.0429 - accuracy: 0.9869
Test Loss: 0.0429
Test Accuracy: 0.9869
